#  Few-shot 프롬프팅

---

# 환경 설정 및 준비

In [ ]:
# Langfuse로 Few-shot 프롬프트 관리하기

from dotenv import load_dotenv
load_dotenv()

import os
import json
from langfuse import get_client
from langfuse.langchain import CallbackHandler
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# Langfuse 설정
langfuse = get_client()
langfuse_handler = CallbackHandler()

# LLM 설정
llm = ChatOpenAI(
    model='gpt-4.1-mini',
    temperature=0.3,
)

# **Zero-shot** 프롬프팅

- **Zero-shot 프롬프팅**은 예시 없이 AI가 즉시 작업을 수행하는 기법입니다

- 명확한 **지시사항**만으로 원하는 결과를 얻을 수 있어 **사용이 간단**합니다

- 단순하고 직관적인 작업에 적합한 프롬프팅 방식이지만, 작업의 **복잡도에 따라 선택적 사용**이 필요합니다

`(1) without context`

In [ ]:
# Zero-shot 프롬프트 생성
langfuse.create_prompt(
    name="competitor-analysis-zero-shot",
    type="text",
    prompt="다음 시장에서 삼성전자의 경쟁업체를 설명해주세요: {{topic}}",
    labels=["production"],
    tags=["competitor", "analysis", "zero-shot"]
)

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Zero-shot 프롬프트 템플릿 사용
zero_shot_template = langfuse.get_prompt("competitor-analysis-zero-shot")

# Langfuse 프롬프트를 LangChain과 통합
zero_shot_prompt = PromptTemplate.from_template(
    zero_shot_template.get_langchain_prompt()
)
zero_shot_prompt.metadata={"langfuse_prompt": zero_shot_template}  # Langfuse 자동 링크를 위한 메타데이터

# 체인 생성
chain = zero_shot_prompt | llm | StrOutputParser()

zero_shot_result = chain.invoke(
    {"topic": "인공지능 반도체"},
    config={"callbacks": [langfuse_handler]}
)

print(zero_shot_result)

`(2) with context`

In [ ]:
# Zero-shot with context 프롬프트 생성 
langfuse.create_prompt(
    name="competitor-analysis-zero-shot-context",
    type="text", 
    prompt="""{{topic}} 시장에서 삼성전자의 경쟁업체를 설명해주세요. 
반드시 다음 제시된 뉴스에 근거해서 답변하세요:

[뉴스]
{{context}}

[답변]
""",
    labels=["production"],
    tags=["competitor", "analysis", "zero-shot", "context"]
)

In [ ]:
# Zero-shot with context 프롬프트 템플릿 사용
zero_shot_template = langfuse.get_prompt("competitor-analysis-zero-shot-context")

# Langfuse 프롬프트를 LangChain과 통합
zero_shot_prompt = PromptTemplate.from_template(
    zero_shot_template.get_langchain_prompt()
)
zero_shot_prompt.metadata={"langfuse_prompt": zero_shot_template}  # Langfuse 자동 링크를 위한 메타데이터


# 체인 생성
chain = zero_shot_prompt | llm | StrOutputParser()

# Zero-shot 실행
context = """삼성전자가 내년 초에 자체적으로 개발한 인공지능(AI) 가속기를 처음으로 출시할 예정이다. 
이는 AI 반도체 시장에서 지배적인 위치를 차지하고 있는 엔비디아의 독점을 도전하고, 
세계 최고의 반도체 제조업체로서의 지위를 다시 확립하려는 삼성전자의 노력으로 해석된다.
"""

topic = "인공지능 반도체"
zero_shot_result = chain.invoke(
    {"context": context, "topic": topic},
    config={"callbacks": [langfuse_handler]}  # Langfuse 트레이싱을 위한 콜백
    )

print(zero_shot_result)

# **One-shot** 프롬프팅

- **One-shot 프롬프팅**은 하나의 예시를 통해 AI가 작업 패턴을 학습하는 기법입니다

- **Zero-shot** 방식보다 더 나은 성능을 제공하며, **형식화된 작업**에 특히 효과적입니다

- 단일 예시로 **품질 향상**이 가능하나, 해당 예시에 **과의존**할 수 있는 한계가 있습니다

In [ ]:
### One-shot 프롬프트 템플릿 생성
# 1. Zero-shot 프롬프트 템플릿에 예시(example)를 포함하도록 수정
# 2. input_variables에 example_topic과 example_response 추가
# 3. config에 예시들을 저장

langfuse.create_prompt(
    name="competitor-analysis-one-shot",
    type="text",
    prompt="""다음은 특정 시장에서 삼성전자의 경쟁업체를 설명하는 예시이다:

시장: {{example_topic}}
경쟁업체: {{example_response}}

이제 다음 시장에서 삼성전자의 경쟁업체를 설명해주세요:
시장: {{topic}}""",
    labels=["production"],
    tags=["competitor", "analysis", "one-shot"],
    config={
        "example_topic": "스마트폰",
        "example_response": "애플: 프리미엄 시장에서 주요 경쟁사로, iPhone 시리즈로 경쟁\n샤오미: 중저가 시장에서 강세를 보이며 글로벌 시장 점유율 확대\n구글: Pixel 시리즈로 프리미엄 시장 진출, AI 기능 강조"
    }
)

In [ ]:
# 프롬프트 템플릿 사용
one_shot_template = langfuse.get_prompt("competitor-analysis-one-shot")

# Langfuse 프롬프트를 LangChain과 통합
one_shot_prompt = PromptTemplate.from_template(
    one_shot_template.get_langchain_prompt()
)
one_shot_prompt.metadata={"langfuse_prompt": one_shot_template}  # Langfuse 자동 링크를 위한 메타데이터

# one_shot_prompt 적용한 체인 생성
chain = one_shot_prompt | llm | StrOutputParser()

# One-shot 실행
topic = "인공지능 반도체"
one_shot_result = chain.invoke(
    input={
        "example_topic": one_shot_template.config.get("example_topic"),
        "example_response": one_shot_template.config.get("example_response"),
        "topic": topic
    },
    config={"callbacks": [langfuse_handler]}  # Langfuse 트레이싱을 위한 콜백
)

print(f"one_shot_result:")
print(one_shot_result)

# **Few-shot** 프롬프팅

- **Few-shot 프롬프팅**은 AI 모델에게 2-5개의 예시를 제공하여 학습시키는 방법입니다

- 이 방식은 **Zero-shot**이나 **One-shot** 프롬프팅보다 더 우수한 성능을 보여주며, 복잡한 작업에서 특히 효과적입니다

- Few-shot 프롬프팅은 높은 성능을 제공하지만, 긴 프롬프트로 인한 **비용 증가**를 고려해야 합니다

In [ ]:
# Example 데이터 준비
examples = """
시장: 스마트폰
경쟁업체: 
- 애플(미국): 프리미엄 시장 주도, iPhone으로 경쟁
- 샤오미(중국): 중저가 시장 강세, 글로벌 확장중
- 구글(미국): Pixel로 AI 기능 강조

시장: TV
경쟁업체:
- LG전자(한국): OLED 기술 경쟁
- Sony(일본): 프리미엄 시장 경쟁
- TCL(중국): 중저가 시장 공략
"""

# Few-shot 프롬프트를 Langfuse에 저장 (예시는 config에 저장)
langfuse.create_prompt(
    name="competitor-analysis-few-shot",
    type="text",
    prompt="""다음은 여러 시장에서 삼성전자의 경쟁업체를 설명하는 예시들이다:

{{examples}}

이제 다음 시장에서 삼성전자의 경쟁업체를 설명해주세요:
시장: {{topic}}""",
    labels=["production"],
    tags=["competitor", "analysis", "few-shot"],
    config={
        "examples": examples
    }
)


In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Few-shot 프롬프트 템플릿 사용
few_shot_template = langfuse.get_prompt("competitor-analysis-few-shot")

# Langfuse 프롬프트를 LangChain과 통합
few_shot_prompt = PromptTemplate.from_template(
    few_shot_template.get_langchain_prompt()
)
few_shot_prompt.metadata={"langfuse_prompt": few_shot_template}  # Langfuse 자동 링크를 위한 메타데이터

# few_shot_prompt 적용한 체인 생성
chain = few_shot_prompt | llm | StrOutputParser()

# Few-shot 실행
topic = "인공지능 반도체"
few_shot_result = chain.invoke(
    input={
        "examples": few_shot_template.config.get("examples"),
        "topic": topic
    },
    config={"callbacks": [langfuse_handler]}  # Langfuse 트레이싱을 위한 콜백
)

print(f"few_shot_result:")
print(few_shot_result)